Loading relevant packages

In [91]:
import requests
import json
from json import loads
import pandas as pd
from datetime import datetime
import time

Using BART API to find relevant station information (Station name, Lattitude & Longitude for mapping)

In [81]:
url = "https://api.bart.gov/api/stn.aspx"  

response = requests.get(url, params={
    "key": key,  # outcoded personal legacy key
    "cmd": "stns",
    "json": "y"
    
})  

time.sleep(2) #maybe use for loop if make mulitple requestst

In [82]:
# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
    print(data)
else:
    print("Error:", response.status_code)

{'?xml': {'@version': '1.0', '@encoding': 'utf-8'}, 'root': {'uri': {'#cdata-section': 'http://api.bart.gov/api/stn.aspx?cmd=stns&json=y'}, 'stations': {'station': [{'name': '12th St. Oakland City Center', 'abbr': '12TH', 'gtfs_latitude': '37.803768', 'gtfs_longitude': '-122.271450', 'address': '1245 Broadway', 'city': 'Oakland', 'county': 'alameda', 'state': 'CA', 'zipcode': '94612'}, {'name': '16th St. Mission', 'abbr': '16TH', 'gtfs_latitude': '37.765062', 'gtfs_longitude': '-122.419694', 'address': '2000 Mission Street', 'city': 'San Francisco', 'county': 'sanfrancisco', 'state': 'CA', 'zipcode': '94110'}, {'name': '19th St. Oakland', 'abbr': '19TH', 'gtfs_latitude': '37.808350', 'gtfs_longitude': '-122.268602', 'address': '1900 Broadway', 'city': 'Oakland', 'county': 'alameda', 'state': 'CA', 'zipcode': '94612'}, {'name': '24th St. Mission', 'abbr': '24TH', 'gtfs_latitude': '37.752470', 'gtfs_longitude': '-122.418143', 'address': '2800 Mission Street', 'city': 'San Francisco', 'co

In [83]:
data = data["root"]["stations"]["station"] #long list of nested dicts, get down
sfstations = [] #initialize empty list of sf stations

In [85]:
for station in data:
    # Check if the city is "San Francisco"
    if station.get("city") == "San Francisco":
        sfstations.append(station)

In [ ]:
df = pd.DataFrame(sfstations)[['name', 'gtfs_latitude', 'gtfs_longitude']]

In [90]:
df # if sf crime api provides info on crime for all bart stations do all bart stations


,name,gtfs_latitude,gtfs_longitude
0,16th St. Mission,37.765062,-122.419694
1,24th St. Mission,37.752470,-122.418143
2,Balboa Park,37.721585,-122.447506
3,Civic Center/UN Plaza,37.779732,-122.414123
4,Embarcadero,37.792874,-122.397020
5,Glen Park,37.733064,-122.433817
6,Montgomery St.,37.789405,-122.401066
7,Powell St.,37.784471,-122.407974
